In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
# Libs
import logging
import pandas as pd
from collections import OrderedDict
import torch

# Own Modules
from source_torch.mlca.mlca import mlca_mechanism>
from source_torch.mlca.mlca_setup import set_value_model_parameters

import datetime
import os
from source_torch.util import save_result, load_result
import numpy as np
import os

MLCA NN Class imported
MLCA NN_MIP Class imported
WDP Class imported
MLCA_Economies imported
MLCA function imported


In [2]:
torch.cuda.set_device(3)
device = torch.device("cuda:3")
iterations = 5
seed = 18
while iterations > 0:
    x = datetime.datetime.now()
    m = x.month
    d = x.day
    h = x.hour
    mi = x.minute

    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    # log debug to console
    logging.basicConfig(level=logging.DEBUG, format='%(asctime)s:               %(message)s', datefmt='%H:%M:%S')
    #logging.basicConfig(level=logging.WARNING, format='%(asctime)s:               %(message)s', datefmt='%H:%M:%S')
    #logging.basicConfig(level=logging.INFO, format='%(asctime)s:               %(message)s', datefmt='%H:%M:%S')    
    
    configdict_mlca = OrderedDict([('SATS_domain_name','GSVM'),
                        ('SATS_auction_instance_seed', seed),
                        ('bidders',0),
                        ('items',0),
                        ('bidder_ids',0),
                        ('Qinit', 30), 
                        ('Qmax', 50),#50
                        ('Qround',0),
                        ('Starter','mlca_extra'),
                        ('epochs' , 512),#512
                        ('batch_size' , 32),
                        ('regularization_type' , 'l1_l2'),
                        ('layer_N' , [10, 10]),
                        ('layer_R' ,[32, 32] ),
                        ('layer_L' , [16, 16]),
                        ('device', device),
                        ('NN_parameters', []),
                        ('bigM',6000),
                        ('warm_start',False),
                        ('MIP_parameters',[]),
                        ('scaler',None),
                        ('init_bids_and_fitted_scaler',[None,None]),
                        ('return_allocation',True),
                        ('return_payments',True),
                        ('lra',0.01),
                        ('regularization',1e-5),
                        ('calc_efficiency_per_iteration',True),
                        ('active_learning_algorithm', 'galo'),
                        ('presampled_n', 20),
                        ('presampled_algorithm', 'unif')])

    configdict_mlca = set_value_model_parameters(configdict_mlca)

    res = mlca_mechanism(configdict = configdict_mlca)
    seed += 1

    if res == 'UNIQUENESS CHECK FAILED see logfile':
        # add one more iteration to the loop
        iterations += 1
        #continue

    iterations -= 1

    dirname = './experiments/MLCA/Torch/results/' + configdict_mlca['SATS_domain_name'] +'_'+ configdict_mlca['active_learning_algorithm']
    result_dir = './experiments/MLCA/Torch/results/' + configdict_mlca['SATS_domain_name'] +'_' + configdict_mlca['active_learning_algorithm'] +'/' +str(m)+'_'+str(d) + '_' + str(h) + '_' + str(mi) + '_' + str(configdict_mlca['SATS_auction_instance_seed'])
    os.makedirs(dirname,exist_ok=True)
    save_result(result_dir, res)

21:05:20:               START MLCA:
21:05:20:               -----------------------------------------------
21:05:20:               Model: GSVM
21:05:20:               Seed SATS Instance: 18
21:05:20:               Qinit: 30
21:05:20:               Qmax: 50
21:05:20:               Qround: 7

21:05:20:               Instantiate SATS Instance



------------------------ SATS parameters ------------------------
Value Model:  GSVM
Number of Bidders:  7
Number of BidderTypes:  2
Number of Items:  18
Scaler:  None

------------------------ DNN  parameters ------------------------
Epochs: 512
Batch Size: 32
Regularization: l1_l2

Bidder_0:
regularization: 1e-05
learning_rate: 0.01
architecture: [32, 32]
dropout: True
dropout_prob: 0.05
epochs: 512
batch_size: 32
regularization_type: l1_l2
device: cuda:3

Bidder_1:
regularization: 1e-05
learning_rate: 0.01
architecture: [32, 32]
dropout: True
dropout_prob: 0.05
epochs: 512
batch_size: 32
regularization_type: l1_l2
device: cuda:3

Bidder_2:
regularization: 1e-05
learning_rate: 0.01
architecture: [32, 32]
dropout: True
dropout_prob: 0.05
epochs: 512
batch_size: 32
regularization_type: l1_l2
device: cuda:3

Bidder_3:
regularization: 1e-05
learning_rate: 0.01
architecture: [32, 32]
dropout: True
dropout_prob: 0.05
epochs: 512
batch_size: 32
regularization_type: l1_l2
device: cuda:3

Bi

21:05:21:               Set NN parameters
21:05:21:               Set MIP parameters
21:05:21:               INITIALIZE BIDS
21:05:21:               -----------------------------------------------

21:05:21:               Set up initial Bids with GALO for: Bidder_0
21:05:21:               Sampling with GALO using unif
21:05:21:               Sampling uniformly at random 20 bundle-value pairs from bidder 0
21:06:04:               Set up initial Bids with GALO for: Bidder_1
21:06:04:               Sampling with GALO using unif
21:06:04:               Sampling uniformly at random 20 bundle-value pairs from bidder 1
21:06:47:               Set up initial Bids with GALO for: Bidder_2
21:06:47:               Sampling with GALO using unif
21:06:47:               Sampling uniformly at random 20 bundle-value pairs from bidder 2
21:07:30:               Set up initial Bids with GALO for: Bidder_3
21:07:30:               Sampling with GALO using unif
21:07:30:               Sampling uniformly at r

First time in optimization step.


21:10:58:               Tighten bounds with IA for Bidder_0
21:10:58:               Tighten bounds with IA for Bidder_2
21:10:58:               Tighten bounds with IA for Bidder_3
21:10:58:               Tighten bounds with IA for Bidder_4
21:10:58:               Tighten bounds with IA for Bidder_5
21:10:58:               Tighten bounds with IA for Bidder_6
21:10:58:               Sorted active bidders in MIP: ['Bidder_0', 'Bidder_2', 'Bidder_3', 'Bidder_4', 'Bidder_5', 'Bidder_6']
21:10:58:               Mip initialized
21:10:58:               Solving MIP
21:10:58:               Attempt no: 1
21:10:58:               Mip time Limit of 1800
21:10:58:               Mip relative gap 0.001
21:10:58:               Mip integrality tol 1e-08


...

Time for MIP run: 39 sec
Found objective 592.7242431911584


22:18:59:               Status  : integer optimal solution
22:18:59:               Time    : 0 sec
22:18:59:               Problem : MILP
22:18:59:               Rel. Gap: 0.0 %
22:18:59:               N. Iter : 31
22:18:59:               Hit Lim.: False
22:18:59:               Objective Value: 581.9571918562315
22:18:59:               ALLOCATION
22:18:59:               ---------------------------------------------
22:18:59:               Calculate MLCA allocation: Main Economy
22:18:59:               Active bidders: ['Bidder_0', 'Bidder_1', 'Bidder_2', 'Bidder_3', 'Bidder_4', 'Bidder_5', 'Bidder_6']
22:18:59:               MLCA allocation in Main Economy:
22:18:59:               Bidder_0 {'good_ids': [], 'value': 0}
22:18:59:               Bidder_1 {'good_ids': [2, 3, 4, 5, 13, 14], 'value': 206.61949129741313}
22:18:59:               Bidder_2 {'good_ids': [], 'value': 0}
22:18:59:               Bidder_3 {'good_ids': [6, 7, 8, 9, 15, 16], 'value': 225.89281091576115}
22:18:59:        


------------------------ SATS parameters ------------------------
Value Model:  GSVM
Number of Bidders:  7
Number of BidderTypes:  2
Number of Items:  18
Scaler:  None

------------------------ DNN  parameters ------------------------
Epochs: 512
Batch Size: 32
Regularization: l1_l2

Bidder_0:
regularization: 1e-05
learning_rate: 0.01
architecture: [32, 32]
dropout: True
dropout_prob: 0.05
epochs: 512
batch_size: 32
regularization_type: l1_l2
device: cuda:3

Bidder_1:
regularization: 1e-05
learning_rate: 0.01
architecture: [32, 32]
dropout: True
dropout_prob: 0.05
epochs: 512
batch_size: 32
regularization_type: l1_l2
device: cuda:3

Bidder_2:
regularization: 1e-05
learning_rate: 0.01
architecture: [32, 32]
dropout: True
dropout_prob: 0.05
epochs: 512
batch_size: 32
regularization_type: l1_l2
device: cuda:3

Bidder_3:
regularization: 1e-05
learning_rate: 0.01
architecture: [32, 32]
dropout: True
dropout_prob: 0.05
epochs: 512
batch_size: 32
regularization_type: l1_l2
device: cuda:3

Bi

22:19:44:               Set up initial Bids with GALO for: Bidder_1
22:19:44:               Sampling with GALO using unif
22:19:44:               Sampling uniformly at random 20 bundle-value pairs from bidder 1
22:20:30:               Set up initial Bids with GALO for: Bidder_2
22:20:30:               Sampling with GALO using unif
22:20:30:               Sampling uniformly at random 20 bundle-value pairs from bidder 2
22:21:12:               Set up initial Bids with GALO for: Bidder_3
22:21:12:               Sampling with GALO using unif
22:21:12:               Sampling uniformly at random 20 bundle-value pairs from bidder 3



KeyboardInterrupt


In [ ]:
iterations = 20
seed = 0

while iterations > 0:
    x = datetime.datetime.now()
    m = x.month
    d = x.day
    h = x.hour
    mi = x.minute

    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    # log debug to console
#    logging.basicConfig(level=logging.DEBUG, format='%(asctime)s:               %(message)s', datefmt='%H:%M:%S')
#    logging.basicConfig(level=logging.WARNING, format='%(asctime)s:               %(message)s', datefmt='%H:%M:%S')
#     logging.basicConfig(level=logging.INFO, format='%(asctime)s:               %(message)s', datefmt='%H:%M:%S')    
    
    configdict_mlca = OrderedDict([('SATS_domain_name','LSVM'),
                        ('SATS_auction_instance_seed', seed),
                        ('bidders',0),
                        ('items',0),
                        ('bidder_ids',0),
                        ('Qinit', 40),
                        ('Qmax', 50),
                        ('Qround',0),
                        ('Starter','mlca_extra'),
                        ('epochs' , 512),#512
                        ('batch_size' , 32),
                        ('regularization_type' , 'l1_l2'),
                        ('layer_N' , [10, 10, 10]),
                        ('layer_R' ,[32, 32] ),
                        ('layer_L' , []),
                        ('device', device),
                        ('NN_parameters',[]),
                        ('bigM',6000),
                        ('warm_start',False),
                        ('MIP_parameters',[]),
                        ('scaler',None),
                        ('init_bids_and_fitted_scaler',[None,None]),
                        ('return_allocation',True),
                        ('return_payments',True),
                        ('lra',0.01),
                        ('regularization',1e-5),
                        ('calc_efficiency_per_iteration',True),
                        ('active_learning_algorithm', 'galo'),
                        ('presampled_n', 20),
                        ('presampled_algorithm', 'unif')])
    configdict_mlca = set_value_model_parameters(configdict_mlca)
    res = mlca_mechanism(configdict = configdict_mlca)
    seed += 1
    iterations -= 1
    if res == 'UNIQUENESS CHECK FAILED see logfile':
        iterations +=1
        continue
    dirname = './experiments/MLCA/Torch/results/' + configdict_mlca['SATS_domain_name'] +'_'+ configdict_mlca['active_learning_algorithm']
    result_dir = './experiments/MLCA/Torch/results/' + configdict_mlca['SATS_domain_name'] +'_' + configdict_mlca['active_learning_algorithm'] +'/' +str(m)+'_'+str(d) + '_' + str(h) + '_' + str(mi) + '_' + str(configdict_mlca['SATS_auction_instance_seed'])
    os.makedirs(dirname,exist_ok=True)
    save_result(result_dir, res)

In [ ]:
results = load_result('/home/artem/ica/experiments/MLCA/Torch/results/GSVM_fft/8_23_12_13_-1')

In [ ]:
# check if all vectors are unique in  results[0]['Elicited Bids']['Bidder_1'][0]
results